## 0. Load datasets

In [1]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sb

# General utils
from tqdm import tqdm
from os import path

from measurements import get_prev_or_next_frame
from sklearn import preprocessing, model_selection, feature_selection, metrics, inspection, pipeline
from sklearn import ensemble, linear_model

def get_balanced_df_by_category(df,category):
    # assert(logical.dtype == bool)
    
    # Find the category with least members
    categories = {cat:mem for cat, mem in df.groupby(category)}
    num_per_category = np.array([len(mem) for mem in categories.values()])
    smallest_category = list(categories.keys())[num_per_category.argmin()]
    
    output = []
    for cat, member in categories.items():
        if cat == smallest_category:
            output.append(member)
        else:
            output.append(member.sample(num_per_category.min()))
    output = pd.concat(output,ignore_index=True)

    return output
    
def predict_feature(df:pd.DataFrame,
                    classifier,
                    feature2predict,
                    Niter:int=100,
                    rebalance:bool=True,
                    subsample=None,
                    measurement_level_name:str='Measurement'):
    if subsample == None:
        subsample = len(df)
    Y_test = []
    Y_pred = []
    models = []
    for i in tqdm(range(Niter)):

        # rng = np.random.RandomState(0)
        df_ = get_balanced_df_by_category(df, df[feature2predict,'Meta'].values)
        
        pipe = pipeline.make_pipeline(preprocessing.StandardScaler(),
                                      classifier() )
        
        y = df_[feature2predict,'Meta'].astype(float)
        X = df_.xs(measurement_level_name,level=1,axis=1).astype(float)
        X[np.isinf(X)] = np.nan
        X_train,X_test,y_train,y_test = model_selection.train_test_split(X,y)
        
        model = pipe.fit(X_train,y_train)
        models.append(model)
        
        Y_test.append(y_test)
        Y_pred.append(model.predict(X_test))
        
    return [np.stack((np.array(Y_test),np.array(Y_pred))),models]
    

In [2]:

dataset_dir = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Mesa et al/Lineage models/Dataset pickles'
all_df = pd.read_pickle(path.join(dataset_dir,f'all_df.pkl'))
divisions = pd.read_pickle(path.join(dataset_dir,f'divisions.pkl'))

# censor the 'answers'
features2drop = [(f,a) for f,a in divisions.columns if 'Time' in f]
features2drop += [(f,a) for f,a in divisions.columns if 'smoothed' in f and not 'rate' in f]
features2drop += [(f,a) for f,a in divisions.columns if 'standard' in f and not 'rate' in f]
features2drop += [(f,a) for f,a in divisions.columns if 'cell coords' in f and not 'rate' in f]
features2drop += ['Z','Z-cyto']
divisions = divisions.drop(columns=features2drop)
print(len(divisions))


668


/var/folders/zp/5k1kvm9j0t19cvq8h628kr8h0000gn/T/ipykernel_52971/312790672.py:11: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  divisions = divisions.drop(columns=features2drop)


In [3]:
divisions

Name          Nuclear volume Nuclear solidity           Y           X  \
Metadata         Measurement      Measurement Measurement Measurement   
Frame TrackID                                                           
0     R1_2          161.5000         0.879211  106.386223   39.835913   
      R1_28         327.5000         0.894656   18.882968   81.480105   
      R1_38         166.1875         0.874383   43.553404   90.039300   
      R1_53         163.1875         0.893566   92.486212   38.868633   
      R1_90         248.6250         0.914903   91.877325   28.223542   
...                      ...              ...         ...         ...   
13    R2_388        199.1875         0.910311   23.771023   16.280515   
      R2_529        234.8125         0.859529   16.878161   25.113388   
      R2_680        249.5625         0.885562   17.805284   89.724017   
      R2_817        215.1250         0.677826  104.740122   35.565369   
      R2_951        220.9375         0.859888   25.618812  103.032815   

Name          Nuclear height Cell volume      Y-cyto      X-cyto  \
Metadata         Measurement Measurement Measurement Measurement   
Frame TrackID                                                      
0     R1_2             -13.0    477.0625  106.461254   40.170935   
      R1_28            -15.0    648.1250   18.734884   81.618756   
      R1_38            -11.0    536.6250   43.781592   89.928081   
      R1_53            -11.0    588.7500   92.722320   38.981768   
      R1_90            -13.0    612.7500   92.214224   28.403458   
...                      ...         ...         ...         ...   
13    R2_388           -10.0    596.3750   23.849691   16.584285   
      R2_529           -11.0    741.7500   16.607137   25.583776   
      R2_680           -10.0    601.8125   17.752337   89.885736   
      R2_817           -12.0    730.2500  104.987847   35.621191   
      R2_951           -11.0    641.6875   25.630393  103.106993   

Name          Axial component Planar component 1  ...  \
Metadata          Measurement        Measurement  ...   
Frame TrackID                                     ...   
0     R1_2           5.422715          22.096147  ...   
      R1_28          8.179093          20.625474  ...   
      R1_38          7.143922          17.733668  ...   
      R1_53          5.573642          28.083831  ...   
      R1_90          7.055729          20.925242  ...   
...                       ...                ...  ...   
13    R2_388         7.833814          16.787922  ...   
      R2_529        11.700021          17.721571  ...   
      R2_680         9.245483          15.434613  ...   
      R2_817         8.398562          26.422073  ...   
      R2_951         8.509350          18.323408  ...   

Name          Relative Nuclear volume at 1 frame prior  \
Metadata                                   Measurement   
Frame TrackID                                            
0     R1_2                                         NaN   
      R1_28                                        NaN   
      R1_38                                        NaN   
      R1_53                                        NaN   
      R1_90                                        NaN   
...                                                ...   
13    R2_388                                  1.188983   
      R2_529                                  1.172629   
      R2_680                                  1.055838   
      R2_817                                  1.543759   
      R2_951                                  0.908046   

Name          Num neighbor division 1 frame prior  \
Metadata                              Measurement   
Frame TrackID                                       
0     R1_2                                    NaN   
      R1_28                                   NaN   
      R1_38                                   NaN   
      R1_53                                   NaN   
      R1_90                              

Name           Nuclear volume  Nuclear solidity           Y           X  \
Frame TrackID                                                             
0     R1_2           161.5000          0.879211  106.386223   39.835913   
      R1_28          327.5000          0.894656   18.882968   81.480105   
      R1_38          166.1875          0.874383   43.553404   90.039300   
      R1_53          163.1875          0.893566   92.486212   38.868633   
      R1_90          248.6250          0.914903   91.877325   28.223542   
...                       ...               ...         ...         ...   
13    R2_388         199.1875          0.910311   23.771023   16.280515   
      R2_529         234.8125          0.859529   16.878161   25.113388   
      R2_680         249.5625          0.885562   17.805284   89.724017   
      R2_817         215.1250          0.677826  104.740122   35.565369   
      R2_951         220.9375          0.859888   25.618812  103.032815   

Name           Nuclear height  Cell volume      Y-cyto      X-cyto  \
Frame TrackID                                                        
0     R1_2              -13.0     477.0625  106.461254   40.170935   
      R1_28             -15.0     648.1250   18.734884   81.618756   
      R1_38             -11.0     536.6250   43.781592   89.928081   
      R1_53             -11.0     588.7500   92.722320   38.981768   
      R1_90             -13.0     612.7500   92.214224   28.403458   
...                       ...          ...         ...         ...   
13    R2_388            -10.0     596.3750   23.849691   16.584285   
      R2_529            -11.0     741.7500   16.607137   25.583776   
      R2_680            -10.0     601.8125   17.752337   89.885736   
      R2_817            -12.0     730.2500  104.987847   35.621191   
      R2_951            -11.0     641.6875   25.630393  103.106993   

Name           Axial component  Planar component 1  ...  \
Frame TrackID                                       ...   
0     R1_2            5.422715           22.096147  ...   
      R1_28           8.179093           20.625474  ...   
      R1_38           7.143922           17.733668  ...   
      R1_53           5.573642           28.083831  ...   
      R1_90           7.055729           20.925242  ...   
...                        ...                 ...  ...   
13    R2_388          7.833814           16.787922  ...   
      R2_529         11.700021           17.721571  ...   
      R2_680          9.245483           15.434613  ...   
      R2_817          8.398562           26.422073  ...   
      R2_951          8.509350           18.323408  ...   

Name           Relative nuc_shcoeffs_L4M2S  Relative nuc_shcoeffs_L4M3S  \
Frame TrackID                                                             
0     R1_2                       -3.781127                     2.084511   
      R1_28                      -4.717312                    77.514036   
      R1_38                      -3.715812                     0.539079   
      R1_53                       0.177180                     3.559932   
      R1_90                      -9.834494                    -0.198583   
...                                    ...                          ...   
13    R2_388                     -0.425296                     0.153937   
      R2_529                      2.158911                     5.049143   
      R2_680                     -0.386701                    -2.156844   
      R2_817                      3.238909                    -7.862416   
      R2_951                     18.872650                     0.658913   

Name           Relative nuc_shcoeffs_L4M4S  Relative nuc_shcoeffs_L5M1S  \
Frame TrackID                                                             
0     R1_2                        1.521338                    -6.081107   
      R1_28                       1.835281                    -3.303854   
      R1_38                       2.201570                    -5.8935

In [25]:
y[I]

Frame  TrackID
0      R1_2        True
       R1_28       True
       R1_38       True
       R1_53       True
       R1_90       True
                  ...  
8      R2_788      True
       R2_813     False
9      R2_619      True
10     R2_25       True
       R2_1063     True
Name: (At least one daughter differentiated, Meta), Length: 303, dtype: object

In [27]:
feature2predict = 'At least one daughter differentiated'

X = divisions.xs('Measurement',axis=1,level=1).dropna(axis=1)
y = divisions[feature2predict,'Meta']

I = ~y.isnull()
X = X[I]
y = y[I].astype(bool)

estimator = ensemble.RandomForestClassifier()

selector = feature_selection.RFECV(estimator,step=1).fit(X,y)

## Recursive feature elimination CV: division

In [30]:
selector.get_feature_names_out()

array(['Nuclear solidity', 'Axial component', 'Planar angle',
       'Basal area', 'Height to BM', 'Mean curvature',
       'cyto_shcoeffs_L1M0C', 'cyto_shcoeffs_L3M0C',
       'cyto_shcoeffs_L5M2C', 'cyto_shcoeffs_L1M1S',
       'cyto_shcoeffs_L3M1S', 'nuc_shcoeffs_L1M0C', 'nuc_shcoeffs_L3M0C',
       'nuc_shcoeffs_L4M4C', 'nuc_shcoeffs_L5M0C', 'nuc_shcoeffs_L3M1S',
       'nuc_shcoeffs_L3M2S', 'nuc_shcoeffs_L3M3S', 'nuc_shcoeffs_L4M4S',
       'nuc_shcoeffs_L5M2S', 'nuc_shcoeffs_L5M4S',
       'nuc_shcoeffs_surface_area', 'Mean adjac Cell volume',
       'Mean adjac Planar component 1', 'Mean adjac Mean curvature',
       'Mean adjac cyto_shcoeffs_L0M0C', 'Mean adjac cyto_shcoeffs_L3M0C',
       'Mean adjac cyto_shcoeffs_L3M2C', 'Mean adjac cyto_shcoeffs_L4M4C',
       'Mean adjac cyto_shcoeffs_L5M2S', 'Mean adjac nuc_shcoeffs_L3M2S',
       'Mean adjac nuc_shcoeffs_L5M4S', 'Mean adjac nuc_shcoeffs_L5M5S',
       'Median adjac Axial angle', 'Median adjac Planar angle',
       'Median

## RFECV: Frame prev to division